In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/bigcontest/bigdata

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import matplotlib as mat

#ignore warnings
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn')


plt.rc('font', family='NanumBarunGothic') 
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf
# import matplotlib as mpl 
# import matplotlib.pyplot as plt 
# import matplotlib.font_manager as fm  

# !apt-get update -qq
# !apt-get install fonts-nanum* -qq

# path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf' 
# font_name = fm.FontProperties(fname=path, size=10).get_name()
# plt.rc('font', family=font_name)

# fm._rebuild()
# mpl.rcParams['axes.unicode_minus'] = False

In [ ]:
data = pd.read_csv('problem_data.csv')

In [ ]:
data.drop('P_TYPE',axis = 1,inplace= True) #어차피 수산물로 다 같기 때문에 필요가 없다.

P_TYPE은 모두 다 같기 때문에 데이터에서 지웠습니다.

In [ ]:
data['VALUE_COUNT'] = 0;
value_dict = {}
for name, value in zip(data['P_NAME'].value_counts().index,data['P_NAME'].value_counts()):
    value_dict[name] = value

def value(col):
    return value_dict[col]

data['VALUE_COUNT'] = data['P_NAME'].apply(value)

어종에 따라서 개수의 차이가 많이 나기 때문에 개수가 작은 걸 처리해주기위해서 P_NAME에 해당하는 value counts를 데이터에 추가해줬습니다. 

In [ ]:
#수입용도 컬럼을 숫자로 바꿔주는 코드
P_purpos = {}
for number,category in enumerate(data['P_PURPOSE'].unique()):
    P_purpos[category] = number

data['P_PURPOSE'] = data['P_PURPOSE'].map(P_purpos)

수입용도를 숫자로 바꿔주는 코드입니다.

In [ ]:
#P_PRICE 평균단가에 ,가 들어간 숫자가 있기에 그 수를 바꿔주고 float 타입으로 다 바꿔줌

def rep(col):
    return col.replace(',','')

data['P_PRICE'] = data['P_PRICE'].map(rep)
data['P_PRICE']= data['P_PRICE'].astype(float)

P_PRICE 데이터 자체에 ,가 들어간 숫자가 한 개 있기에 ,를 바꿔주고
기본 데이터 타입이 object로 되어 있어서 float으로 변환해줬습니다.

In [ ]:
#제조국과 수출국에 있는 모든 도시들을 합쳐서 숫자로 바꿔줌
city_list = []

for i in data['CTRY_1'].unique():
    city_list.append(i)

for i in data['CTRY_2'].unique():
    if i not in city_list:
        city_list.append(i)

citytonumber = {}
for number,city in enumerate(city_list):
    citytonumber[city] = number

data['CTRY_1'] = data['CTRY_1'].map(citytonumber)
data['CTRY_2'] = data['CTRY_2'].map(citytonumber)

제조국과 수출국에 있는 모든 도시들을 합쳐서
숫자로 바꿔줬습니다.

In [ ]:
#종분류명 카테고리와 어종을 숫자로 바꿔주는 코드

ca1 = {}

for idx,name in enumerate(data['CATEGORY_1'].unique()):
    ca1[name] = idx;

ca2 = {}

for idx,name in enumerate(data['CATEGORY_2'].unique()):
    ca2[name] = idx;

data['CATEGORY_1'] = data['CATEGORY_1'].map(ca1)
data['CATEGORY_2'] = data['CATEGORY_2'].map(ca2)

종분류명과 어종을 숫자로 바꿔주는 코드입니다.

In [ ]:
#P_IMPORT_TYPE 수입형태를 숫자로 바꿔주는 코드

p_type = {}
for idx, name in enumerate(data.P_IMPORT_TYPE.unique()):
    p_type[name] = idx

data['P_IMPORT_TYPE'] = data['P_IMPORT_TYPE'].map(p_type)

P_IMPORT_TYPE을 숫자로 바꿔주는 코드입니다.

In [ ]:
#P_NAME 상세어종을 숫자로 바꿔주는 코드

p_name = {}
for idx, name in enumerate(data.P_NAME.unique()):
    p_name[name] = idx
data['P_NAME'] = data['P_NAME'].map(p_name)

상세어종을 숫자로 바꿔주는 코드입니다.

In [ ]:
#강수량 데이터를 기존에 있던 데이터에 있는 날짜에 맞게 바꿔주는 코드

rain = pd.read_csv('강수량.csv',encoding = 'cp949')

rainday = rain['날짜']
rainamount = rain['강수량(mm)']

raindict = {}

for day,amount in zip(rainday,rainamount):
    raindict[day] = amount

data['RAIN'] = 0

def rain(col):
    return raindict[col]

data['RAIN'] = data['REG_DATE'].apply(rain)

어획량 자체에 강수량이 영향을 끼친다고 판단을 해서 기존의 데이터에 강수량 데이터를 추가해주는 코드입니다.

In [ ]:
#소비자물가지수 데이터를 기존의 데이터에 추가하고 날짜에 맞게 값을 넣어주는 코드

consumer = pd.read_csv('소비자물가지수.csv',encoding = 'cp949')
consumer.drop('시도별',axis =1 , inplace = True)
consumer.columns = consumer.columns.str.replace('.','-')
consumerday = consumer.columns
consumerday = consumerday.str.replace(' ','')
consumeramount = consumer.iloc[0]

consumerdict = {}
for day,amount in zip(consumerday,consumeramount):
    consumerdict[day] = amount


data['CONSUMER'] = 0

def consumersplit(col):
    a = col.split('-')
    b = '-'.join(a[:2])
    return consumerdict[b]

data['CONSUMER'] = data['REG_DATE'].apply(consumersplit)

소비자 물가지수를 통해서 가격을 유추할 수 있다고 판단을 해 소비자물가지수 데이터를 기존의 데이터에 추가했습니다.

In [ ]:
#기온 데이터를 기존의 데이터에 추가하고 날짜에 맞게 값을 넣어주는 코드

data_tem = pd.read_csv('기온.csv',encoding='cp949')
data['TEMP'] = 0

tem_day = data_tem['날짜']
tem = data_tem['평균기온(℃)']

tem_dict = {}

for day, t in zip(tem_day, tem):
  tem_dict[day] = t

data['TEMP'] = data['REG_DATE'].map(tem_dict)

어획량과 가격에 기온이 영향을 미친다고 생각해서 기온데이터를 기존 데이터에 추가했습니다.

In [ ]:
#인구 데이터를 기존의 데이터에 추가하고 날짜에 맞게 값을 넣어주는 코드

data_popul_1 = pd.read_csv('201512_201712.csv',encoding = 'cp949')
data_popul_2 = pd.read_csv('201801_201912.csv',encoding = 'cp949')
data_popul_1 = data_popul_1.iloc[0,1::4]
data_popul_2 = data_popul_2.iloc[0,1::4]
data_popul = pd.concat([data_popul_1,data_popul_2])


data_popul.index = data_popul.index.str.replace('_총인구수', '')
data_popul.index = data_popul.index.str.replace('년', '-')
data_popul.index = data_popul.index.str.replace('월', '')

data_popul_dict = {}

for day,amount in zip(data_popul.index,data_popul):
    data_popul_dict[day] = amount

def populsplit(col):
    a = col.split('-')
    b = '-'.join(a[:2])
    return data_popul_dict[b]

data['POPULATION'] = 0

data['POPULATION'] = data['REG_DATE'].apply(populsplit)
data['POPULATION'] = data['POPULATION'].str.replace(',','')
data['POPULATION'] = data['POPULATION'].astype(int)

인구가 가격에 영향을 미친다고 생각해 기존의 데이터에 추가했습니다.

In [ ]:
# #모델을 돌리기 위해서 REG_DATE를 year,month,day로 나눠주는 코드

# def getyear(col):
#     return col.split('-')[0]

# def getmonth(col):
#     return col.split('-')[1]

# def getday(col):
#     return col.split('-')[2]

# data['YEAR'] = 0
# data['MONTH'] = 0
# data['DAY'] = 0

# data['YEAR'] = data['REG_DATE'].apply(getyear)
# data['MONTH'] = data['REG_DATE'].apply(getmonth)
# data['DAY'] = data['REG_DATE'].apply(getday)

# data['YEAR'] = data['YEAR'].astype(int)
# data['MONTH'] = data['MONTH'].astype(int)
# data['DAY'] = data['DAY'].astype(int)


REG_DATE를 year, month, day로 나눠주는 코드입니다.
REG_DATE가 object로 되어 모델에 넣을 수 없기에 해줬습니다.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostRegressor

target = data['P_PRICE']
train_data = data.drop(['P_PRICE','REG_DATE'],axis = 1)

x_train,x_test,y_train,y_test = train_test_split(train_data,target,random_state = 0)

model = RandomForestRegressor(n_estimators=500,random_state=0)
model.fit(x_train,y_train)
cross = cross_val_score(model,x_test,y_test,cv = 5)

print('cross_tree',cross.mean())



예측해야되는 값이 P_PRICE를 target에 넣고 train_data에 P_PRICE와 REG_DATE를 drop하고 넣어줍니다. train_test_split을 이용해서 데이터를 나눠준 뒤 RandomForestREgressor를 진행했습니다. cross_val_score까지 하니 약 88%의 정확도가 나왔습니다.

In [ ]:
from sklearn.svm import SVR
model = SVR()
model.fit(x_train,y_train)
cross_svr = cross_val_score(model,x_test,y_test,cv = 5)
print('cross_svr',cross_svr.mean())

SVR을 이용하는 코드입니다.

In [ ]:
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=500,learning_rate=0.05,max_depth=3)
model.fit(x_train,y_train)
cross_xgboost = cross_val_score(model,x_test,y_test,cv = 5)

print('cross_xgboost : ',cross_xgboost.mean())

XGBRegressor를 이용하는 코드입니다.

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(x_train,y_train)

cross_linear = cross_val_score(model,x_test,y_test,cv = 5)
print(cross_linear)
print('cross_linear : ',cross_linear.mean())

LinearRegression을 이용하는 코드입니다.

In [ ]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor().fit(x_train,y_train)
cross_MLP = cross_val_score(model,x_test,y_test,cv = 5)

print('croos_MLP : ',cross_MLP.mean())

MLPREGREESOR을 이용하는 코드입니다.

In [ ]:
# data['CTRY_1'] = data['CTRY_1'].astype('category')
# data['CTRY_2'] = data['CTRY_2'].astype('category')
# data['P_PURPOSE'] = data['P_PURPOSE'].astype('category')
# data['CATEGORY_1'] = data['CATEGORY_1'].astype('category')
# data['CATEGORY_2'] = data['CATEGORY_2'].astype('category')
# data['P_NAME'] = data['P_NAME'].astype('category')
# data['P_IMPORT_TYPE'] = data['P_IMPORT_TYPE'].astype('category')

In [ ]:
heatmap_data = data[['CTRY_2','P_PURPOSE','CATEGORY_1','CATEGORY_2','P_NAME','P_IMPORT_TYPE','P_PRICE','RAIN',
                     'TEMP']]

colormap = plt.cm.RdBu
plt.figure(figsize=(14, 12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(heatmap_data.astype(float).corr(), linewidths=0.1, vmax=1.0,
           square=True, cmap=colormap, linecolor='white', annot=True, annot_kws={"size": 11})


각 feature들 간의 상관관계를 보기 위해서 heatmap_data를 그렸습니다.

In [ ]:
#data.drop(['REG_DATE','CTRY_1','CONSUMER','POPULATION'],axis = 1,inplace = True)

In [ ]:
# fig ,ax = plt.subplots(1,1,figsize = (20,12))
# sns.scatterplot('P_NAME','P_PRICE',data = data,ax = ax)
# plt.xlim(0,20)

In [ ]:
# fig,ax = plt.subplots(3,1,figsize = (20,20));
# sns.scatterplot('P_NAME','P_PRICE',data = data,ax = ax[0])
# ax[0].set_xlim(0,51)
# ax[0].set_xticks(range(0,51));

# sns.scatterplot('P_NAME','P_PRICE',data = data,ax = ax[1])
# ax[1].set_xlim(51,101)
# ax[1].set_xticks(range(51,101));

# sns.scatterplot('P_NAME','P_PRICE',data = data,ax = ax[2])
# ax[2].set_xlim(101,141)
# ax[2].set_xticks(range(101,141));


In [ ]:
fig,ax = plt.subplots(3,1,figsize = (20,20));
sns.scatterplot('P_NAME','P_PRICE',data = data,ax = ax[0])
ax[0].set_xlim(141,180)
ax[0].set_xticks(range(141,180));
# plt.xlim(0,20)

sns.scatterplot('P_NAME','P_PRICE',data = data,ax = ax[1])
ax[1].set_xlim(180,230)
ax[1].set_xticks(range(180,230));
# plt.xlim(0,20)

sns.scatterplot('P_NAME','P_PRICE',data = data,ax = ax[2])
ax[2].set_xlim(230,285)
ax[2].set_xticks(range(230,286));
# plt.xlim(0,20)

P_NAME과 P_PRICE를 이용해서 scatterplot을 그렸습니다.

In [ ]:
#아웃라이어 없애기
def outlier_iqr(data):
    q1,q3 = np.percentile(data,[25,75])
    iqr = q3-q1
    lower_bound = q1 - (iqr * 1.5)
    upper_bound = q3 + (iqr * 1.5)

    return np.where((data > upper_bound) | (data < lower_bound))

아웃라이어를 없애기 위한 함수입니다.

In [ ]:
# index = []
# for i in range(0,286):
#     if(data[data['P_NAME'] == i]['P_PRICE'].shape[0]<= 5):
#         continue
#     a = list(outlier_iqr(data[data['P_NAME'] == i]['P_PRICE']))
#     index.append(a)


# for idx in index:
#     data.drop(idx,inplace = True)

In [ ]:
# grouped_df = data.groupby(['P_NAME'])

# for key, item in grouped_df:
#     print(grouped_df.get_group(key), "\n\n")